<a href="https://colab.research.google.com/github/vitroid/GenIce/blob/genice3/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

On Google Colaboratory, you must install GenIce2 and extra plugins by yourself.


In [ ]:
try:
    import google.colab
    %pip install git+https://github.com/vitroid/GenIce.git@genice2 git+https://github.com/vitroid/genice2-svg.git
except:
    pass


In [ ]:
! pip install cif2ice

## Output in Gromacs format.

A simplest example.


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter, Molecule

genice = GenIce3()
genice.unitcell = UnitCell("ice11")
Exporter("gromacs", water_model=Molecule("spce")).dump(genice)

You can also invoke the plugins directly, instead of loading via the loaders such as `UnitCell` or `Exporter`.


In [ ]:
from genice3.genice import GenIce3
from genice3.exporter.gromacs import dump
from genice3.unitcell.ice11 import UnitCell as ice11

genice = GenIce3(unitcell=ice11())
dump(genice, water_model="4site")

## Generate an SVG image

Here we are using the Forms syntax of Google Colab to choose the ice type.


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
from logging import getLogger, basicConfig, INFO
from IPython.display import SVG, display

ice = "DDR"  # @param ["ice1h", "ice2", "ice3", "DDR"] {allow-input: true}

basicConfig(level=INFO)
genice = GenIce3()
genice.unitcell = UnitCell(ice)
svg = Exporter("svg").dumps(genice, shadow=True, rotate="y5,x5")
display(SVG(svg))

## Generate a PNG image

PNG renderer is also included in genice_svg package.


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
from logging import getLogger, basicConfig, INFO
from IPython.display import SVG, display


basicConfig(level=INFO)
genice = GenIce3()
genice.unitcell = UnitCell("CS2")
genice.replication_matrix = [[2, 0, 0], [0, 2, 0], [0, 0, 2]]
png = Exporter("png").dumps(
    genice, shadow="#8882", encode=False, bgcolor="#fff", rotate="y5,x5"
)
png

## Store the image in a file

Save it in a PNG file.


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
from logging import getLogger, basicConfig, INFO

basicConfig(level=INFO)
genice = GenIce3()
genice.unitcell = UnitCell("ice3")
genice.replication_matrix = [[2, 0, 0], [0, 2, 0], [0, 0, 2]]
with open("ice3.png", "wb") as f:
    Exporter("png").dump(genice, f, shadow="#8882", bgcolor="#fff", rotate="y5,x5")

## Analysis

Some statistical analyses are also available.


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
import numpy as np
from matplotlib import pyplot as plt
import random

genice = GenIce3(replication_matrix=[[2,0,0],[0,2,0],[0,0,2]])
genice.unitcell = UnitCell("ice1c")
for seed in range(1000, 1010):
    genice.seed = seed
    kg = Exporter("_KG").calculate(genice)
    plt.plot(kg[:, 0], kg[:, 1])

plt.xlabel("r / nm")
plt.ylabel("Kirkwood G")
plt.show()

## Logging

You may want to see the process log.


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
from logging import getLogger, INFO, DEBUG, WARNING, basicConfig

basicConfig(level=INFO)
logger = getLogger()

genice = GenIce3(replication_matrix=[[16, 0, 0], [0, 16, 0], [0, 0, 16]])
genice.unitcell = UnitCell("ice1c")
gro = Exporter("gromacs", water_model="spce").dumps(genice)
with open("ice1c.gro", "w") as f:
    f.write(gro)

logger.setLevel(WARNING)

## Raw data

You can access the "raw" (internal) data.


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell

genice = GenIce3()
genice.unitcell = UnitCell("xFAU", rep=3)
genice.lattice_sites

ユーザーが参照できる変数、設定できるReactiveな変数を列挙する。

### Visualization by Plotly

Use the raw data to plot by yourself.


In [ ]:
waters = genice.lattice_sites  # cached; no additional calculation
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=waters[:, 0],
            y=waters[:, 1],
            z=waters[:, 2],
            marker=dict(size=10),
            mode="markers",
        )
    ]
)
fig.show()

## Integration with py3Dmol.

Py3Dmol works on Google Colaboratory.


In [ ]:
try:
    import google.colab
    %pip install git+https://github.com/vitroid/genice3-py3dmol.git
except:
    pass


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter

genice = GenIce3(unitcell=UnitCell("ice1h_unit"))
Exporter("py3dmol").get_view(genice, water_model="4site").show()

## Interaction with MDAnalysis

| genice3 を膨張させたくないので、MDAnalysis のような大きなパッケージは external plugin に押しだす。


In [ ]:
try:
    import google.colab
    %pip install git+https://github.com/vitroid/genice3-mdanalysis.git
except:
    pass

In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter

genice = GenIce3(unitcell=UnitCell("ice1h_unit"))
universe = Exporter("mdanalysis").get_universe(genice, water_model="4site")
# Save as a .pdb file.
allatoms = universe.select_atoms("all")
allatoms.write("1h_unit.pdb")